In [1]:
# Import necessary libraries
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Input
from keras.layers.noise import AlphaDropout
from keras.layers import Layer
from tensorflow.python.keras import backend as K
from sklearn.model_selection import train_test_split
import pickle

# Import custom modules
from network import *
from data import *

dbfile = open('../Data/full', 'rb')
data = pickle.load(dbfile)
dbfile = open('../Data/location_pert', 'rb')
location_pert = pickle.load(dbfile)
dbfile = open('../Data/pert2profiles', 'rb')
pert2profiles = pickle.load(dbfile)
dbfile = open('../Data/test_perts', 'rb')
test_pert = pickle.load(dbfile)
dbfile = open('../Data/train_perts', 'rb')
train_pert = pickle.load(dbfile)
dbfile.close()

# X_train = generate_data(data,train_pert,100)
# X_test = generate_data(data,test_pert,100)

# dbfile = open('X_train_triplet_full', 'ab')
# pickle.dump(X_train, dbfile)
# dbfile.close()


#X = pickle.load(open('../Data/X_train_triplet_full', 'rb'))
#test = pickle.load(open('../Data/X_test_triplet_full', 'rb'))
# y = pickle.load(open('../Data/y_test', 'rb'))

# cross validation code WIP
all_pert = np.concatenate((train_pert, test_pert))

from sklearn.model_selection import KFold

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# cross validation code WIP
all_pert = np.concatenate((train_pert,test_pert))

from sklearn.model_selection import KFold
def cross_validate(splitsize):
    kf = KFold(n_splits=splitsize)
    results = []
    c=1
    for train_idx, val_idx in kf.split(all_pert):
        print("\nCrossvalidation run #",c)
        c+=1
        train_perts = all_pert[train_idx]
        val_perts = all_pert[val_idx]
        print(train_perts.shape,val_perts.shape)
        X_train = generate_data(data,train_perts,100)
        print("\n")
        X_val = generate_data(data,val_perts,100)
        s = siamese("cos","net")
        epochs=10
        print("\n")
        embeddings, trained, pred, p_loss, n_loss, train_acc_l, test_acc_l = run_network(s, epochs, X_train, X_val)
        
        p = np.sum(trained[0][2]<=0.5)
        n = np.sum(trained[0][3]>0.5)
        train_acc = ((p+n)/len(X_train[0])/2)

        p = np.sum(pred[0][2]<=0.5)
        n = np.sum(pred[0][3]>0.5)
        val_acc = ((p+n)/len(X_val[0])/2)
        
        accuracy = (train_acc, val_acc)
        results.append(accuracy)
    return results

result = cross_validate(5)
test_res=0
train_res=0
for i in result:
    test_res+=i[1]
    train_res+=i[0]
print ("\n\nCross-validation result: Train: \n%s\t\t Test:%s" % (train_res,test_res))


Crossvalidation run # 1
(1736,) (434,)
batch_size:  10
10/10

batch_size:  10
10/10

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Epoch			+ Dist	- Dist		Train	Test
Epoch 0:	10/10	0.244	0.327		85.000	70.000
Epoch 1:	10/10	0.372	0.797		95.000	55.000
Epoch 2:	10/10	0.071	0.798		95.000	70.000
Epoch 3:	10/10	0.030	0.880		95.000	40.000
Epoch 4:	10/10	0.021	0.943		100.000	40.000
Epoch 5:	10/10	0.019	0.996		100.000	45.000
Epoch 6:	10/10	0.004	1.000		100.000	50.000
Epoch 7:	10/10	0.002	0.999		100.000	50.000
Epoch 8:	10/10	0.001	0.999		100.000	50.000
Epoch 9:	10/10	0.001	0.999		100.000	50.000

Crossvalidation run # 2
(1736,) (434,)
batch_size:  10
10/10

batch_size:  10
10/10

Initialized
Epoch			+ Dist	- Dist		Train	Test
Epoch 0:	10/10	0.139	0.142		70.000	50.000
Epoch 1:	10/10	0.447	0.701		90.000	75.000
Epoch 2:	10/10	0.199	0.920		100.000	65.000
Epoch 3:	10/10	0.078	0.927		100.000	75.000
Epoch 4:	10/10	0.036	0.954		100.000	80.000
Epoch 5:	10/10	0.058	0.